In [1]:
import matplotlib.pyplot as plt

In [2]:
import numpy as np

In [3]:
from datetime import datetime
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.layers import LeakyReLU
from datetime import datetime
from keras import optimizers
from keras import backend

In [4]:
# from keras.utils import plot_model
import time
from keras.utils.vis_utils import plot_model

In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('aisdata-2022-03-22.csv', skipinitialspace=True)
# df = pd.read_csv('/run/ChenAISData/US_AIS_201912_7days_1282656rows_filted.csv', skipinitialspace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11720315 entries, 0 to 11720314
Data columns (total 26 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   # Timestamp                     object 
 1   Type of mobile                  object 
 2   MMSI                            int64  
 3   Latitude                        float64
 4   Longitude                       float64
 5   Navigational status             object 
 6   ROT                             float64
 7   SOG                             float64
 8   COG                             float64
 9   Heading                         float64
 10  IMO                             object 
 11  Callsign                        object 
 12  Name                            object 
 13  Ship type                       object 
 14  Cargo type                      object 
 15  Width                           float64
 16  Length                          float64
 17  Type of position fixing d

In [8]:
df.fillna(0)  

,# Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Length,Type of position fixing device,Draught,Destination,ETA,Data source type,A,B,C,D
0,22/03/2022 00:00:00,Class A,265031000,55.066410,12.725533,Under way using engine,0.0,12.8,208.2,203.0,...,0.0,Undefined,0.0,0,0,AIS,0.0,0.0,0.0,0.0
1,22/03/2022 00:00:00,Class A,219022188,55.716850,12.631587,Under way using engine,0.0,9.0,52.4,54.0,...,0.0,Undefined,0.0,0,0,AIS,0.0,0.0,0.0,0.0
2,22/03/2022 00:00:00,AtoN,992111751,54.540190,11.281527,Unknown value,0.0,0.0,0.0,0.0,...,1.0,GPS,0.0,0,0,AIS,0.0,1.0,0.0,1.0
3,22/03/2022 00:00:00,Class A,266262000,54.368367,11.979833,Under way using engine,0.0,16.2,60.2,61.0,...,0.0,Undefined,0.0,0,0,AIS,0.0,0.0,0.0,0.0
4,22/03/2022 00:00:00,Class A,219016781,54.572652,11.929285,Under way using engine,0.0,0.0,303.2,131.0,...,0.0,Undefined,0.0,0,0,AIS,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11720310,22/03/2022 23:59:58,Class A,232019804,55.932133,2.837362,Under way using engine,0.0,10.7,61.3,62.0,...,84.0,Internal,5.9,NEU MUKRAN,25/03/2022 18:00:00,AIS,20.0,64.0,10.0,10.0
11720311,22/03/2022 23:59:58,Class A,211141000,55.543333,7.365333,Under way using engine,0.0,15.7,351.0,350.0,...,134.0,GPS,7.4,DKSKA,23/03/2022 13:00:00,AIS,124.0,10.0,9.0,14.0
11720312,22/03/2022 23:59:58,Class A,209356000,55.020027,4.261000,Under way using engine,-1.1,19.5,179.6,179.0,...,299.0,GPS,12.0,FOR ORDERS,23/03/2022 15:30:00,AIS,220.0,79.0,31.0,19.0
11720313,22/03/2022 23:59:58,Class A,219026706,55.544758,12.702945,Under way using engine,0.0,10.4,275.3,320.0,...,15.0,GPS,1.5,0,0,AIS,12.0,3.0,2.0,3.0


In [9]:
# get rid of nan rows (in speed and course) - could just set to -1
df = df.dropna() 


In [10]:
# change dataframe to numpy a
df = df.values

In [11]:
print(df)

[['22/03/2022 00:00:03' 'Class A' 235006758 ... 8.0 4.0 4.0]
 ['22/03/2022 00:00:04' 'Class A' 477537200 ... 20.0 3.0 27.0]
 ['22/03/2022 00:00:04' 'Class A' 636012383 ... 43.0 16.0 26.0]
 ...
 ['22/03/2022 23:59:58' 'Class A' 248738000 ... 15.0 4.0 10.0]
 ['22/03/2022 23:59:58' 'Class A' 219455000 ... 97.0 15.0 15.0]
 ['22/03/2022 23:59:58' 'Class A' 209356000 ... 79.0 31.0 19.0]]


In [16]:

n_rows, n_cols = df.shape


df = df[np.lexsort((df[:, 0], df[:, 2]))]

In [17]:
print("row=%d,col=%d"%(n_rows,n_cols)) 
print(df)

row=1559147,col=26
[['22/03/2022 00:02:50' 'Class A' 205038000 ... 57.0 7.0 9.0]
 ['22/03/2022 00:05:06' 'Class A' 205038000 ... 57.0 7.0 9.0]
 ['22/03/2022 00:08:09' 'Class A' 205038000 ... 57.0 7.0 9.0]
 ...
 ['22/03/2022 23:59:38' 'Class A' 664444000 ... 31.0 16.0 16.0]
 ['22/03/2022 23:59:46' 'Class A' 664444000 ... 31.0 16.0 16.0]
 ['22/03/2022 23:59:57' 'Class A' 664444000 ... 31.0 16.0 16.0]]


In [18]:

s_time=time.mktime( time.strptime('22/03/2022 00:00:00', "%d/%m/%Y %H:%M:%S" )) # 转换成时间类型)
              
for i in range(n_rows):
    b_time=time.mktime( time.strptime(df[i][0], "%d/%m/%Y %H:%M:%S") )
    df[i][0] =(int)(b_time-s_time)#int(df[i][0][11:13])*3600 + int(df[i][0][14:16])*60 + int(df[i][0][17:])
 

In [19]:
np.savez('20220322_sorted_data.npz', sorted_data=df)

In [20]:
print(df)
print(df.shape)  

[[170 'Class A' 205038000 ... 57.0 7.0 9.0]
 [306 'Class A' 205038000 ... 57.0 7.0 9.0]
 [489 'Class A' 205038000 ... 57.0 7.0 9.0]
 ...
 [86378 'Class A' 664444000 ... 31.0 16.0 16.0]
 [86386 'Class A' 664444000 ... 31.0 16.0 16.0]
 [86397 'Class A' 664444000 ... 31.0 16.0 16.0]]
(1559147, 26)


In [21]:
print('----------------------------------------------------')
print('End of danish_pull_data.py')
print('----------------------------------------------------')

----------------------------------------------------
End of danish_pull_data.py
----------------------------------------------------
